In [7]:
# set desired output FPS
FPS_OUTPUT = 2
import pandas as pd
import numpy as np
import cv2
import sys
import os
from datetime import datetime

# setup paths
pwd = os.getcwd().replace("notebooks","")
path_data = pwd + 'data/'

path_labels_file = pwd + 'outdoor_label_timestamps.csv'
if not os.path.exists(path_labels_file):
    print("ERROR - cannot find input timestamps label file")
    
def get_seconds(minute,second):
    ts = datetime.now().replace(hour=0, minute=minute, second = second, microsecond = 0)
    return int((ts - ts.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds())
def get_seconds_start(x):
    return get_seconds(x[4],x[5])

def get_seconds_end(x):
    return get_seconds(x[6],x[7])
df = pd.read_csv(path_labels_file)

# extract components of start and end times
df['start_minute'] = df['time_start'].str.split(":").str.get(1).astype(int)
df['start_second'] = df['time_start'].str.split(":").str.get(2).astype(int)

df['end_minute'] = df['time_end'].str.split(":").str.get(1).astype(int)
df['end_second'] = df['time_end'].str.split(":").str.get(2).astype(int)

df.head()

KeyError: 'time_start'

In [9]:
df['start_elapsed'] = df.apply(get_seconds_start, axis=1)
df['end_elapsed'] = df.apply(get_seconds_end, axis=1)

del df['time_start']
del df['time_end']
del df['start_minute']
del df['start_second']
del df['end_minute']
del df['end_second']

df.head()

IndexError: index out of bounds

In [10]:
videos = list(df['video'].unique())
# initialize output
out_all = []

for video in videos:
    # subset dataframe to labels for this video
    dft = df[df['video'] == video]

    out = []

    for i, row in enumerate(dft.values):
        vid = row[0]
        label = row[1]
        start = row[2]
        end = row[3]

        for i in range(start,end):
            for j in range(FPS_OUTPUT):
                out.append({"video":vid,
                            "frame": vid + "_{:06d}".format(len(out)) + '.jpg',
                            "label": label})
    
    out_all.extend(out)

TypeError: 'str' object cannot be interpreted as an integer

In [11]:
# make dataframe and save
df_out = pd.DataFrame(out_all)
df_out = df_out[['video','frame','label']]
df_out.to_csv(path_data + 'labels.csv',index=False)

print("SAVED LABELS FILE to /data/labels.csv")

KeyError: "None of [Index(['video', 'frame', 'label'], dtype='object')] are in the [columns]"